In [1]:
import pandas as pd
import numpy as np
import pymongo

In [2]:
# Save path to data set in a variable (.tsv files)
title_basics_file = "../Resources/title_basics.tsv"
title_akas_file = "../Resources/title_akas.tsv"
title_principals_file = "../Resources/title_principals.tsv"
title_ratings_file = "../Resources/title_ratings.tsv"
name_basics_file = "../Resources/name_basics.tsv"


In [3]:
# Read the original data files (.tsv files)
title_basics_df= pd.read_csv(title_basics_file, sep='\t')
title_basics_df

C:\Users\ben\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
6848584,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
6848585,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
6848586,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
6848587,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [4]:
##Clean up content, drop adult movies
title_basics_clean = title_basics_df.loc[title_basics_df['isAdult']==0]

##Drop tv episodes
title_basics_clean = title_basics_clean.loc[title_basics_clean['titleType']=='movie']

##Drop unnecessary columns (isAdult)
title_basics_clean = title_basics_clean.drop(['isAdult'], axis=1)
title_basics_clean = title_basics_clean.drop(['endYear'], axis=1)
title_basics_clean = title_basics_clean.drop(['originalTitle'], axis=1)

##Rename columns
title_basics_clean = title_basics_clean.rename(columns={"tconst":"titleId","titleType":"Type"})

#Replace \N w/ NaN
title_basics_clean = title_basics_clean.replace('\\N', 'N\A')

#title_basics_clean.drop_index
#title_basics_clean = title_basics_clean.set_index(["titleId"])

title_basics_clean

,titleId,Type,primaryTitle,startYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,1894,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport"
332,tt0000335,movie,Soldiers of the Cross,1900,N\A,"Biography,Drama"
499,tt0000502,movie,Bohemios,1905,100,N\A
571,tt0000574,movie,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama"
...,...,...,...,...,...,...
6848478,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,57,Documentary
6848505,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,2007,100,Documentary
6848517,tt9916706,movie,Dankyavar Danka,2013,N\A,Comedy
6848528,tt9916730,movie,6 Gunn,2017,116,N\A


In [5]:
# Read the title_principals_file data files (.tsv files)
title_principals_df= pd.read_csv(title_principals_file, sep='\t')
title_principals_df

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
39502495,tt9916880,5,nm0996406,director,principal director,\N
39502496,tt9916880,6,nm1482639,writer,\N,\N
39502497,tt9916880,7,nm2586970,writer,books,\N
39502498,tt9916880,8,nm1594058,producer,producer,\N


In [6]:
##Rename columns
title_principals_df = title_principals_df.rename(columns={"tconst":"titleId","nconst":"nameId"})

##Drop unnecessary columns (ordering)
title_principals_df = title_principals_df.drop(['ordering','job'], axis=1)

#Replace \N w/ N\A
title_principals_df = title_principals_df.replace('\\N', 'N\A')

title_principals_df = title_principals_df.loc[title_principals_df['category'].str.contains("actor|actress|director")]

#title_basics_clean.drop_index
#title_principals_df = title_principals_df.set_index(["titleId"])

title_principals_df

,titleId,nameId,category,characters
1,tt0000001,nm0005690,director,N\A
3,tt0000002,nm0721526,director,N\A
5,tt0000003,nm0721526,director,N\A
9,tt0000004,nm0721526,director,N\A
11,tt0000005,nm0443482,actor,"[""Blacksmith""]"
...,...,...,...,...
39502492,tt9916880,nm0254176,actress,"[""Moody Margaret""]"
39502493,tt9916880,nm0286175,actor,"[""Dad"",""Aerobic Al"",""Nasty Nicola""]"
39502494,tt9916880,nm10535738,actress,"[""Horrid Henry""]"
39502495,tt9916880,nm0996406,director,N\A


In [7]:
# Read the original data files (.tsv files)
title_ratings_df= pd.read_csv(title_ratings_file, sep='\t')
title_ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.6,1619
1,tt0000002,6.0,198
2,tt0000003,6.5,1301
3,tt0000004,6.1,122
4,tt0000005,6.1,2078
...,...,...,...
1045450,tt9916576,6.0,9
1045451,tt9916578,8.5,18
1045452,tt9916720,5.7,52
1045453,tt9916766,6.8,13


In [8]:
##Rename columns
title_ratings_df = title_ratings_df.rename(columns={"tconst":"titleId"})

#title_basics_clean.drop_index
#title_ratings_df = title_ratings_df.set_index(["titleId"])

title_ratings_df

,titleId,averageRating,numVotes
0,tt0000001,5.6,1619
1,tt0000002,6.0,198
2,tt0000003,6.5,1301
3,tt0000004,6.1,122
4,tt0000005,6.1,2078
...,...,...,...
1045450,tt9916576,6.0,9
1045451,tt9916578,8.5,18
1045452,tt9916720,5.7,52
1045453,tt9916766,6.8,13


In [9]:
# Read the original data files (.tsv files)
name_basics_df= pd.read_csv(name_basics_file, sep='\t')
name_basics_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0050419,tt0043044,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0038355,tt0037382"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,producer","tt0054452,tt0059956,tt0049189,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0078723,tt0077975,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050976,tt0050986,tt0083922,tt0060827"
...,...,...,...,...,...,...
10127930,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10127931,nm9993716,Essias Loberg,\N,\N,NaN,\N
10127932,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10127933,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [10]:
##Rename columns
name_basics_df = name_basics_df.rename(columns={"nconst":"nameId"})

##Drop unnecessary columns (ordering)
name_basics_df = name_basics_df.drop(['birthYear','deathYear','knownForTitles','primaryProfession'], axis=1)


#title_basics_clean.drop_index
#name_basics_df = name_basics_df.set_index(["nameId"])

name_basics_df

,nameId,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
...,...,...
10127930,nm9993714,Romeo del Rosario
10127931,nm9993716,Essias Loberg
10127932,nm9993717,Harikrishnan Rajan
10127933,nm9993718,Aayush Nair


In [11]:
#Replace \N w/ N\A
name_basics_df = name_basics_df.replace('\\N', 'N\A')

name_basics_df = name_basics_df.fillna('missing')

##Clean up content, drop knowForTitles='N\A'
#name_basics_df = name_basics_df.loc[name_basics_df['assocTitleId']!='N\A']
##name_basics_df = name_basics_df.loc[name_basics_df['primaryProfession']!='N\A']

name_basics_df

,nameId,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman
...,...,...
10127930,nm9993714,Romeo del Rosario
10127931,nm9993716,Essias Loberg
10127932,nm9993717,Harikrishnan Rajan
10127933,nm9993718,Aayush Nair


In [12]:
## Merge title_basics_clean, title_ratings_df
merge_title_df = pd.merge(title_basics_clean, title_ratings_df, on="titleId", how="inner")
merge_title_df

,titleId,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,1894,45,Romance,5.9,154
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,347
2,tt0000335,movie,Soldiers of the Cross,1900,N\A,"Biography,Drama",6.1,40
3,tt0000502,movie,Bohemios,1905,100,N\A,3.8,6
4,tt0000574,movie,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,574
...,...,...,...,...,...,...,...,...
245073,tt9915790,movie,Bobbyr Bondhura,2019,N\A,Family,7.4,16
245074,tt9916132,movie,The Mystery of a Buryat Lama,2018,94,"Biography,Documentary,History",3.2,6
245075,tt9916160,movie,Drømmeland,2019,72,Documentary,6.6,36
245076,tt9916428,movie,The Secret of China,2019,N\A,"Adventure,History,War",3.3,11


In [13]:
#####
# Now need to merge the dataframes together into one dataframe
# with all the required data
####

title_principals_df.head(5)


,titleId,nameId,category,characters
1,tt0000001,nm0005690,director,N\A
3,tt0000002,nm0721526,director,N\A
5,tt0000003,nm0721526,director,N\A
9,tt0000004,nm0721526,director,N\A
11,tt0000005,nm0443482,actor,"[""Blacksmith""]"


In [14]:
merge_title_df.head(5)

,titleId,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,movie,Miss Jerry,1894,45,Romance,5.9,154
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,347
2,tt0000335,movie,Soldiers of the Cross,1900,N\A,"Biography,Drama",6.1,40
3,tt0000502,movie,Bohemios,1905,100,N\A,3.8,6
4,tt0000574,movie,The Story of the Kelly Gang,1906,70,"Biography,Crime,Drama",6.1,574


In [15]:
name_basics_df.head(5)

,nameId,primaryName
0,nm0000001,Fred Astaire
1,nm0000002,Lauren Bacall
2,nm0000003,Brigitte Bardot
3,nm0000004,John Belushi
4,nm0000005,Ingmar Bergman


In [16]:
## Merge merge_title_df and title_principals_df
merge_title_df2 = pd.merge(title_principals_df, merge_title_df, on="titleId", how="inner")
merge_title_df2

,titleId,nameId,category,characters,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000009,nm0063086,actress,"[""Miss Geraldine Holbrook (Miss Jerry)""]",movie,Miss Jerry,1894,45,Romance,5.9,154
1,tt0000009,nm0183823,actor,"[""Mr. Hamilton""]",movie,Miss Jerry,1894,45,Romance,5.9,154
2,tt0000009,nm1309758,actor,"[""Chauncey Depew - the Director of the New Yor...",movie,Miss Jerry,1894,45,Romance,5.9,154
3,tt0000009,nm0085156,director,N\A,movie,Miss Jerry,1894,45,Romance,5.9,154
4,tt0000147,nm0714557,director,N\A,movie,The Corbett-Fitzsimmons Fight,1897,20,"Documentary,News,Sport",5.2,347
...,...,...,...,...,...,...,...,...,...,...,...
1181703,tt9916538,nm8678236,actress,"[""Sinta""]",movie,Kuambil Lagi Hatiku,2019,123,Drama,8.4,5
1181704,tt9916538,nm1417182,actress,"[""Widhi""]",movie,Kuambil Lagi Hatiku,2019,123,Drama,8.4,5
1181705,tt9916538,nm10041459,actor,"[""Vikash""]",movie,Kuambil Lagi Hatiku,2019,123,Drama,8.4,5
1181706,tt9916538,nm1266058,actress,"[""Dewi""]",movie,Kuambil Lagi Hatiku,2019,123,Drama,8.4,5


In [17]:
## Merge merge_title_df2 and name_basics_df
movie_df = pd.merge(merge_title_df2, name_basics_df, on="nameId", how="inner")
movie_df

,titleId,nameId,category,characters,Type,primaryTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,primaryName
0,tt0000009,nm0063086,actress,"[""Miss Geraldine Holbrook (Miss Jerry)""]",movie,Miss Jerry,1894,45,Romance,5.9,154,Blanche Bayliss
1,tt0000009,nm0183823,actor,"[""Mr. Hamilton""]",movie,Miss Jerry,1894,45,Romance,5.9,154,William Courtenay
2,tt0019859,nm0183823,actor,"[""Cyril Wimborne""]",movie,Evidence,1929,N\A,"Drama,Romance",7.1,14,William Courtenay
3,tt0020403,nm0183823,actor,"[""The Minister - Guillotine Sequence""]",movie,Show of Shows,1929,128,Musical,5.9,334,William Courtenay
4,tt0000009,nm1309758,actor,"[""Chauncey Depew - the Director of the New Yor...",movie,Miss Jerry,1894,45,Romance,5.9,154,Chauncey Depew
...,...,...,...,...,...,...,...,...,...,...,...,...
1181083,tt9916428,nm9445072,actor,"[""Mao Ze Dong""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",3.3,11,Wang Peng Kai
1181084,tt9916428,nm8594703,actor,"[""Dr. Hatem""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",3.3,11,Valery Gadreau
1181085,tt9916428,nm8680851,actor,"[""Powell Editor-in-chief of US newspaper""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",3.3,11,Vincent Matile
1181086,tt9916428,nm2631291,actor,"[""lee De""]",movie,The Secret of China,2019,N\A,"Adventure,History,War",3.3,11,A.C Stokes


In [18]:
movie_final_df = movie_df[['primaryName','primaryTitle','category','characters','startYear','runtimeMinutes','genres','averageRating','numVotes']]

movie_final_df = movie_final_df.rename(columns={"primaryName":"Name","primaryTitle":"movieTitle","category":"role"})

movie_final_df

,Name,movieTitle,role,characters,startYear,runtimeMinutes,genres,averageRating,numVotes
0,Blanche Bayliss,Miss Jerry,actress,"[""Miss Geraldine Holbrook (Miss Jerry)""]",1894,45,Romance,5.9,154
1,William Courtenay,Miss Jerry,actor,"[""Mr. Hamilton""]",1894,45,Romance,5.9,154
2,William Courtenay,Evidence,actor,"[""Cyril Wimborne""]",1929,N\A,"Drama,Romance",7.1,14
3,William Courtenay,Show of Shows,actor,"[""The Minister - Guillotine Sequence""]",1929,128,Musical,5.9,334
4,Chauncey Depew,Miss Jerry,actor,"[""Chauncey Depew - the Director of the New Yor...",1894,45,Romance,5.9,154
...,...,...,...,...,...,...,...,...,...
1181083,Wang Peng Kai,The Secret of China,actor,"[""Mao Ze Dong""]",2019,N\A,"Adventure,History,War",3.3,11
1181084,Valery Gadreau,The Secret of China,actor,"[""Dr. Hatem""]",2019,N\A,"Adventure,History,War",3.3,11
1181085,Vincent Matile,The Secret of China,actor,"[""Powell Editor-in-chief of US newspaper""]",2019,N\A,"Adventure,History,War",3.3,11
1181086,A.C Stokes,The Secret of China,actor,"[""lee De""]",2019,N\A,"Adventure,History,War",3.3,11


In [19]:
movie_final_df = movie_final_df.loc[movie_final_df['numVotes']>89999]
movie_final_df



,Name,movieTitle,role,characters,startYear,runtimeMinutes,genres,averageRating,numVotes
1452,Michael Curtiz,Casablanca,director,N\A,1942,102,"Drama,Romance,War",8.5,503468
1819,Victor Sjöström,Wild Strawberries,actor,"[""Dr. Eberhard Isak Borg""]",1957,91,"Drama,Romance",8.2,90925
3248,Alfred Abel,Metropolis,actor,"[""Johann (Joh) Fredersen""]",1927,153,"Drama,Sci-Fi",8.3,153870
3524,Harry Myers,City Lights,actor,"[""An Eccentric Millionaire""]",1931,87,"Comedy,Drama,Romance",8.5,159568
7544,Charles Chaplin,The Kid,actor,"[""A Tramp""]",1921,68,"Comedy,Drama,Family",8.3,106681
...,...,...,...,...,...,...,...,...,...
1173599,Vilhelm Blomgren,Midsommar,actor,"[""Pelle""]",2019,148,"Drama,Horror,Mystery",7.1,155708
1175011,Bryon Lerum,Extraction,actor,"[""Rake's Son""]",2020,116,"Action,Thriller",6.8,114245
1175012,Ryder Lerum,Extraction,actor,"[""Rake's Son""]",2020,116,"Action,Thriller",6.8,114245
1175013,Sam Hargrave,Extraction,director,N\A,2020,116,"Action,Thriller",6.8,114245


In [20]:
#movie_final_df1 = movie_final_df.iloc[0:399999]
#movie_final_df2 = movie_final_df.iloc[400000:799999]
#movie_final_df3 = movie_final_df.iloc[800000:]

movie_final_df.to_json(r'movies_votes90k.json')
#movie_final_df2.to_json(r'movies2.json')
#movie_final_df3.to_json(r'movies3.json')

In [21]:
## setup mongo connection
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'moviesDB2' database in Mongo
db = client.moviesDB2

In [22]:
## panda dataframe insert to mongo for each dataframe:
##     merge_title_df
#takes about 3 min to load

movies = db.movies
movies.insert_many(movie_final_df.to_dict('records'))
#movies.insert_many(movie_final_df2.to_dict('records'))
#movies.insert_many(movie_final_df3.to_dict('records'))


In [56]:
#movies.insert_many(movie_final_df.to_dict('records'))


In [23]:
#movies.insert_many(movie_final_df3.to_dict('records'))

In [24]:
## panda dataframe insert to mongo for each dataframe:
##     name_basics_df
#takes about 5 min to load

#names = db.names
#names.insert_many(name_basics_df.to_dict('records'))

In [25]:
## panda dataframe insert to mongo for each dataframe:
##     title_principals_df
#divide into 4 sections with rows to prevent breaking our computers

In [26]:
#title_principals_df_1 = title_principals_df.iloc[0:9000000]
#principals = db.principals

#principals.insert_many(title_principals_df_1.to_dict('records'))

In [27]:
#title_principals_df_2 = title_principals_df.iloc[9000001:18000000]

#principals.insert_many(title_principals_df_2.to_dict('records'))

In [28]:
#title_principals_df_3 = title_principals_df.iloc[18000001:27000000]

#principals.insert_many(title_principals_df_3.to_dict('records'))

In [29]:
#title_principals_df_4 = title_principals_df.iloc[27000001:]

#principals.insert_many(title_principals_df_4.to_dict('records'))
